In [1]:
import pyedflib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.fftpack import rfft, irfft, fftfreq
import scipy as sp
from scipy import signal

import os
import itertools

from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV

%matplotlib inline

In [2]:
HOME = "/Users/jonat/OneDrive/Documents/Protocolo_PKS/Protocolo_PKS/dados_edf/DB_REPO/edf/"
def dadosPK(tipo):
    dir_ct = HOME+tipo+"/"
    return os.listdir(HOME+tipo+"/")
def carregarDataFrame(tipo, caminho):
    caminho = HOME+tipo+"/"+caminho
    
    edf = pyedflib.EdfReader(caminho)
    n = edf.signals_in_file
    sigbufs = np.zeros((n, edf.getNSamples()[0]))
    for i in np.arange(n):
         sigbufs[i, :] = edf.readSignal(i)
    edf._close()
    del edf

    return pd.DataFrame(data=sigbufs.T, columns=['ch1', 'ch2', 'ch3', 'ch4'])

In [3]:
def plot_confusion_matrix(cm, classes, normalize=True, title='Matriz de confusão, sem normalização', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(12,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Esperado')
    plt.xlabel('Obtido')

In [4]:
def fft(df, canal):
    amostras = int(df[canal][2000:11000].shape[0]*500/2000)
    sinal = np.abs(np.fft.fft(df[canal][2000:11000]))[:amostras]
    freq = np.linspace(0,500,amostras)
    return (sinal,freq)

def frequecia(df, canal):
    sinalFFT = [fft(data, canal)[0] for data in df]
    return sinalFFT

def plotarFFT(a, b, titulo):
    plt.title(titulo)
    plt.plot(b,a)
    plt.xlim([0,500])
    plt.ylim([0,5000])
    plt.show

In [5]:
df_ct = [ carregarDataFrame('controle', nome) for nome in dadosPK('controle')]
df_pk = [ carregarDataFrame('parkinson', nome) for nome in dadosPK('parkinson')]

In [8]:
sinalCT = frequecia(df_ct, 'ch1')
sinalPK = frequecia(df_pk, 'ch1')

rotulosCT = [0 for _ in sinalCT]
rotulosPK = [1 for _ in sinalPK]

trainData = sinalCT + sinalPK
rotulos = rotulosCT + rotulosPK

In [7]:
#32,91, 100, 9,12, 22, 25, 34, 46, 62, 63,65,69,85,
param_grid = [
    {'C': [1], 'gamma': [0.1], 'kernel':['poly'], 'degree':[4,3]}
]
nomes = ['controle','parkinson']
rangeA = [32,91, 100, 9,12, 22, 25, 34, 46, 62, 63,65,69,85]

for i in rangeA:
    X_train, X_test, y_train, y_test = train_test_split(trainData, rotulos, test_size=0.3, random_state=i)
    grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    print(str(i)+"#")
    print(classification_report(y_pred, y_test,target_names=nomes))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] C=1, degree=4, gamma=0.1, kernel=poly ...........................
[CV] . C=1, degree=4, gamma=0.1, kernel=poly, score=0.8, total=   0.0s
[CV] C=1, degree=4, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=4, gamma=0.1, kernel=poly, score=0.7368421052631579, total=   0.0s
[CV] C=1, degree=4, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=4, gamma=0.1, kernel=poly, score=0.631578947368421, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.75, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.7368421052631579, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.6842105263157895, total=   0.0s
32#
             precision    reca

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished


[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.8947368421052632, total=   0.0s
100#
             precision    recall  f1-score   support

   controle       0.79      0.92      0.85        12
  parkinson       0.92      0.79      0.85        14

avg / total       0.86      0.85      0.85        26

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] C=1, degree=4, gamma=0.1, kernel=poly ...........................
[CV] . C=1, degree=4, gamma=0.1, kernel=poly, score=0.5, total=   0.0s
[CV] C=1, degree=4, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=4, gamma=0.1, kernel=poly, score=0.7894736842105263, total=   0.0s
[CV] C=1, degree=4, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=4, gamma=0.1, kernel=poly, score=0.7368421052631579, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV] . C=1, degree=3, gamma=0.1, kernel=poly, score=0.5, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=po

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished


[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.75, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.6842105263157895, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.6842105263157895, total=   0.0s
22#
             precision    recall  f1-score   support

   controle       0.83      0.83      0.83        12
  parkinson       0.86      0.86      0.86        14

avg / total       0.85      0.85      0.85        26

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] C=1, degree=4, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=4, gamma=0.1, kernel=poly, score=0.75, total=   0.0s
[CV] C=1, degree=4, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=4, gamma=0.1, kernel=poly, score=0.7368421052631579, total=   0.0s
[CV] C=1, degree=4, gamma=0.1, kernel=pol

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished


[CV]  C=1, degree=4, gamma=0.1, kernel=poly, score=0.5789473684210527, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.65, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.9473684210526315, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.5263157894736842, total=   0.0s
46#
             precision    recall  f1-score   support

   controle       0.73      0.92      0.81        12
  parkinson       0.91      0.71      0.80        14

avg / total       0.83      0.81      0.81        26

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] C=1, degree=4, gamma=0.1, kernel=poly ...........................
[CV] . C=1, degree=4, gamma=0.1, kernel=poly, score=0.7, total=   0.0s
[CV] C=1, degree=4, gamma=0.1, kernel=pol

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished


[CV]  C=1, degree=4, gamma=0.1, kernel=poly, score=0.6842105263157895, total=   0.0s
[CV] C=1, degree=4, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=4, gamma=0.1, kernel=poly, score=0.6842105263157895, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.65, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.7368421052631579, total=   0.0s
[CV] C=1, degree=3, gamma=0.1, kernel=poly ...........................
[CV]  C=1, degree=3, gamma=0.1, kernel=poly, score=0.6842105263157895, total=   0.0s
65#
             precision    recall  f1-score   support

   controle       0.67      0.75      0.71         8
  parkinson       0.88      0.83      0.86        18

avg / total       0.82      0.81      0.81        26

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] C=1, degree=4, gamma=0

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
